<a href="https://colab.research.google.com/github/YasirHabib/Deep-Learning-Advanced-Computer-Vision/blob/master/resnet_first_layers_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install import-ipynb
import import_ipynb
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Copy the link and remove the front part of the link (i.e. https://drive.google.com/open?id=) to get the file ID.
your_module = drive.CreateFile({'id':'1VwKqcBIeZjyeEGfC7oVd_kifRAoPoMNP'})
your_module.GetContentFile('conv_block_tf.ipynb')
from conv_block_tf import ConvLayer, BatchNormLayer, ConvBlock

In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import keras

from keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions

In [0]:
# define some additional layers so they have a forward function
class ZeroPaddingLayer:
  def __init__(self, padding):
    self.padding=padding

  def forward(self, X):
    return tf.keras.layers.ZeroPadding2D(padding=self.padding)(X)

  def get_params(self):
    return []

class ReLULayer:
  def forward(self, X):
    return tf.nn.relu(X)

  def get_params(self):
    return []

class MaxPoolLayer:
  def __init__(self, dim):
    self.dim = dim

  def forward(self, X):
    return tf.nn.max_pool(
      X,
      ksize=[1, self.dim, self.dim, 1],
      strides=[1, 2, 2, 1],
      padding='VALID'
    )

  def get_params(self):
    return []

In [0]:
class PartialResNet:
  def __init__(self):
    # before conv block
    self.layers=[
        ZeroPaddingLayer(padding=3),
        ConvLayer(64, 3, 7, 7, stride=2, padding='VALID'),
        BatchNormLayer(64),
        ReLULayer(),
        ZeroPaddingLayer(padding=1),
        MaxPoolLayer(dim=3),
        ConvBlock(num_color_channels=64, fm_sizes=[64, 64, 256], stride=1)
    ]
    
    self.input_ = tf.placeholder(tf.float32, shape=(None, 224, 224, 3))
    self.output = self.forward(self.input_)

  def copyFromKerasLayers(self, layers):
    self.layers[1].copyFromKerasLayers(layers[2])
    self.layers[2].copyFromKerasLayers(layers[3])
    self.layers[6].copyFromKerasLayers(layers[7:17])

  def forward(self, X):
    for layer in self.layers:
      X = layer.forward(X)
      print(X)
    return X

  def predict(self, X):
    assert(self.session is not None)
    return self.session.run(
      self.output,
      feed_dict={self.input_: X}
    )

  def set_session(self, session):
    self.session = session
    self.layers[1].session = session
    self.layers[2].session = session
    self.layers[6].set_session(session)

  def get_params(self):
    params = []
    for layer in self.layers:
      params += layer.get_params()

In [10]:
if __name__ == '__main__':
  # you can also set weights to None, it doesn't matter
  resnet = ResNet50(weights='imagenet')
  #resnet.summary()      to print all the resnet layers

  partial_model = Model(
    inputs=resnet.input,
    outputs=resnet.layers[18].output
  )
  print(partial_model.summary())       # to print only the resnet layers based on [18] in the above line
  # for layer in partial_model.layers:
  #   layer.trainable = False
  
  # print(partial_model.layers)
  # [<keras.engine.input_layer.InputLayer object at 0x7f3132aee208>, 
  #  <keras.layers.convolutional.ZeroPadding2D object at 0x7f3132aeea90>,
  #  <keras.layers.convolutional.Conv2D object at 0x7f3132aee9e8>,
  #  <keras.layers.normalization.BatchNormalization object at 0x7f3132aeeba8>,
  #  <keras.layers.core.Activation object at 0x7f3132aeec18>, 
  #  <keras.layers.convolutional.ZeroPadding2D object at 0x7f312edf9ac8>, 
  #  <keras.layers.pooling.MaxPooling2D object at 0x7f312ed99710>, 
  #  <keras.layers.convolutional.Conv2D object at 0x7f312eddf5f8>, 
  #  <keras.layers.normalization.BatchNormalization object at 0x7f312ecde0b8>, 
  #  <keras.layers.core.Activation object at 0x7f312ecdef98>, 
  #  <keras.layers.convolutional.Conv2D object at 0x7f312ecedc88>, 
  #  <keras.layers.normalization.BatchNormalization object at 0x7f312ec86f60>, 
  #  <keras.layers.core.Activation object at 0x7f312ec46a20>, 
  #  <keras.layers.convolutional.Conv2D object at 0x7f312ebc2cf8>, 
  #  <keras.layers.convolutional.Conv2D object at 0x7f312ebb0748>, 
  #  <keras.layers.normalization.BatchNormalization object at 0x7f312ebf3710>, 
  #  <keras.layers.normalization.BatchNormalization object at 0x7f312ea45d68>, 
  #  <keras.layers.merge.Add object at 0x7f312ea13c88>, 
  #  <keras.layers.core.Activation object at 0x7f312ea2fe80>]


  my_partial_resnet = PartialResNet()

  # make a fake image
  X = np.random.random((1, 224, 224, 3))

  # get keras output
  keras_output = partial_model.predict(X)

  # get my model output
  init = tf.variables_initializer(my_partial_resnet.get_params())

  # note: starting a new session messes up the Keras model
  session = keras.backend.get_session()
  my_partial_resnet.set_session(session)
  session.run(init)

  # first, just make sure we can get any output
  first_output = my_partial_resnet.predict(X)
  print("first_output.shape:", first_output.shape)

  # copy params from Keras model
  my_partial_resnet.copyFromKerasLayers(partial_model.layers)

  # compare the 2 models
  output = my_partial_resnet.predict(X)
  diff = np.abs(output - keras_output).sum()
  if diff < 1e-10:
    print("Everything's great!")
  else:
    print("diff = %s" % diff)

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________